# Чтение данных

In [32]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [33]:
df_list = []

for i in range(12):
    df_list.append(
        pd.read_csv(f"columns/out_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )
df = pd.concat(df_list, axis=0)
df

Skipping line 49611: '⇧' expected after '"'
Skipping line 49612: '⇧' expected after '"'
Skipping line 49613: '⇧' expected after '"'
Skipping line 346135: field larger than field limit (131072)


,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,Конструктор
1,dataset/Dataset/data/3693708/table_0,1,Двигатель
2,dataset/Dataset/data/3693708/table_0,2,Сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,Лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,Очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,Дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,Сезон
415129,dataset/Dataset/data/6206410/table_0,1,Короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


# Research
---
Проблемные столбцы (скорее всего не будем использовать):
- Столбцы по типу: "ΔHкип, кДж·моль−1"
- Столбцы использующиеся для группировки каких-либо значений по времени (месяцу, году и т.п.)
- Столбцы без имени
- Столбцы, название которых не имеет смысла (числа и т.п.)
- Аббревиатуры и сокращения

### Столбцы без названия = 164500
их можно отправить в тестовую выборку, чтобы не терять информацию. (нужно еще посмотреть, что внутри них обычно лежит)

### Без имени
можно в тест

## .mv-parser... = 15661
обрезать только осмысленный текст

---

In [34]:
df["column_name"].value_counts()

column_name
Год                         220591
Место                        87172
Роль                         78947
Команда                      67696
Название                     64944
                             ...  
COM Express / ETXexpress         1
Различныхтопологий               1
РазличныхТ0 топологий            1
Неэквивалентнымтопологий         1
На обсервации                    1
Name: count, Length: 125691, dtype: int64

## Столбцы без названия

In [35]:
unnamed_columns = df["column_name"].apply(lambda x: str(x).startswith("Unnamed"))
df[unnamed_columns]

,table_id,column_id,column_name
92,dataset/Dataset/data/7901984/table_1,11,Unnamed: 11
199,dataset/Dataset/data/5929754/table_7,1,Unnamed: 1
200,dataset/Dataset/data/5929754/table_7,2,Unnamed: 2
201,dataset/Dataset/data/5929754/table_7,3,Unnamed: 3
202,dataset/Dataset/data/5929754/table_7,4,Unnamed: 4
...,...,...,...
414659,dataset/Dataset/data/5676641/table_9,1,Unnamed: 1
414782,dataset/Dataset/data/8352540/table_0,1,Unnamed: 1
414824,dataset/Dataset/data/1447471/table_0,1,Unnamed: 1
415023,dataset/Dataset/data/2708389/table_2,0,Unnamed: 0


## Столбцы содержащие в названии: ".mw-parser..."

In [36]:
mv_parser = df["column_name"].apply(lambda x: str(x).strip().startswith(".mw-parser"))
df[mv_parser]

,table_id,column_id,column_name
339,dataset/Dataset/data/3595819/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...
374,dataset/Dataset/data/6412392/table_1,2,.mw-parser-output .ts-comment-commentedText{bo...
646,dataset/Dataset/data/5220971/table_15,1,.mw-parser-output .ts-comment-commentedText{bo...
1159,dataset/Dataset/data/6614435/table_0,2,.mw-parser-output .ts-comment-commentedText{bo...
1277,dataset/Dataset/data/7406863/table_0,7,.mw-parser-output .ts-comment-commentedText{bo...
...,...,...,...
413067,dataset/Dataset/data/4558392/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...
413415,dataset/Dataset/data/7720725/table_1,9,.mw-parser-output .ts-comment-commentedText{bo...
413645,dataset/Dataset/data/309979/table_2,1,.mw-parser-output .ts-comment-commentedText{bo...
413832,dataset/Dataset/data/3100187/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...


### В назнваниях столбца содержится человекопонятное название, нужно отчистить мусорную информацию

In [37]:
mv_parser_humnareadable_names = df[mv_parser]["column_name"].apply(lambda x: str(x).split("}")[-1])
mv_parser_humnareadable_names.unique()

array(['М', 'МУ', 'Д / В', 'П', 'В', '№', 'П.', '7,5+7,5 км', 'И', 'Ист.',
       'Л/Круги', 'Коэфф.', 'Г. р.', 'Жертвы', 'Кубок', 'З', 'Портрет',
       'Поз.', 'Г', 'WR', 'Год', 'Категория', '99/00', 'Лучший новичок',
       'Игр', 'ХП', 'No.', 'Прим.', 'Индив.', 'Уров.', 'Сб', 'ПП',
       'Церемония', 'ЧС', 'КП', 'Коэф.', 'Bib', 'Поул', 'Ист', '11/12',
       'СС', 'Л.круги', 'ОС МСОП', '04/05', 'PPG', 'Этап', 'ПСЛ', 'Поз',
       'Пб', 'СУ', 'ЧР', 'Л/круги', '09/10', ' И', 'GP', '+', 'Код',
       'Субъект РФ', 'ОП', 'Место', 'Дата рождения', 'КЧ', '12/13', 'О',
       'ТТХ', 'Итого', '15+15 км', 'И.', 'Многоборье', 'Дом.', 'Кат',
       'Пр.', 'Численностьверующихмлн. человек', 'Разница', 'МГ', '10/11',
       'Голосовало', 'Country Songs[A]', 'Пал.', 'Σ', '#', 'APG', 'G',
       'Ном.', 'Б/Н', '+/- (в\xa0%)', '№ п/п', 'Период', 'Отчёт', 'Н',
       'иностр.', 'Д / В / Н', 'Pos.', 'Итог[1]', 'Пик', 'Релиз', 'SPG',
       'СУ, км', 'ГЗ', 'СР', 'Сн.', 'Победа', 'КН', ' В', 'Зав. №'

## Аббревиатуры / сокращения / меры (кг, кдж.)

In [38]:
# TODO: what should i do? -_-

## Заголовки, состоящие из одной буквы

In [39]:
no_word_in_header = df["column_name"].str.len() < 2
df[no_word_in_header]["column_name"].unique()

array(['№', 'И', 'В', 'П', 'О', '#', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', 'Н', '±', '%', '+', '−', '=', 'Ш', 'Б', 'Л', 'М', '-',
       'С', 'З', 'Г', 'N', '«', '»', 'm', 'V', 'G', 'W', 'L', 'А', 'Д',
       '0', 'Р', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l',
       'n', 'X', 'ə', 'u', 'o', 'H', 'J', 'K', 'I', 'C', 'D', '—', 'Т',
       'F', '¬', 'M', 'z', 'Х', 'У', '’', 'Э', 'Z', 'R', 'с', '學', 'ä',
       'Δ', 'E', '<', '>', 'T', 'A', 'O', 'S', 'U', 'Y', 'P', 'B', 'Q',
       'p', '*', 'Ч', 't', 'Ф', 'χ', 'Σ', '↕', 's', 'Ц', 'н', '×', 'К',
       '₣', 'j', 'γ', '.', ':', 'ʤ', 'w', '·', 'м', 'α', 'β', 'δ', 'ε',
       'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'σ',
       'τ', 'υ', 'φ', 'ψ', 'ω', 'ḥ', 'r', 'ḳ', 'ḫ', 'Á', 'Ⱥ', 'Ć', 'Ȼ',
       'ʼ', 'Е', '…', '將', '張', 'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ',
       'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ', '•', 'q', '$', 'v', 'x', 'y', 'Ж',
       '←', '↔', '→', 'Ә', 'Ғ', 'Ё', 'Й', '꤀', '꤁', '꤂', '꤃', '꤄

# Очистка данных

---
1. Очистка мусора из столбцов, содержащих ".mv-parser.."
2. Выделение столбцов без заголовка
3. Замена специальных символов на слова (№, $, %, и т.п.)
4. Очистка символов не из русского алфавита
5. Удаление лишних пробелов с начала и конца заголовка
---

In [40]:
df_cleansed = df.copy()

## 1. Очистка мусора из столбцов

In [41]:
def clean_mv_parser(col_name: str) -> str:
    if col_name.strip().startswith(".mw-parser"):
        return col_name.split("}")[-1]
    return col_name

df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: clean_mv_parser(str(x)))

100%|████████████████████████████| 4965678/4965678 [00:02<00:00, 2368682.05it/s]


In [42]:
df_cleansed[mv_parser]

,table_id,column_id,column_name
339,dataset/Dataset/data/3595819/table_0,0,М
374,dataset/Dataset/data/6412392/table_1,2,М
646,dataset/Dataset/data/5220971/table_15,1,МУ
1159,dataset/Dataset/data/6614435/table_0,2,Д / В
1277,dataset/Dataset/data/7406863/table_0,7,П
...,...,...,...
413067,dataset/Dataset/data/4558392/table_0,0,М
413415,dataset/Dataset/data/7720725/table_1,9,В
413645,dataset/Dataset/data/309979/table_2,1,В
413832,dataset/Dataset/data/3100187/table_0,0,М


## 2. Выделение столбцов без заголовка

In [43]:
unnamed_columns = df_cleansed["column_name"].str.startswith("Unnamed")
df_cleansed = df_cleansed[~unnamed_columns]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,Конструктор
1,dataset/Dataset/data/3693708/table_0,1,Двигатель
2,dataset/Dataset/data/3693708/table_0,2,Сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,Лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,Очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,Дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,Сезон
415129,dataset/Dataset/data/6206410/table_0,1,Короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3. Замена специальных символов на слова

In [44]:
def remove_alphabetic_symbols(col_name: str) -> str:
    """TODO: just replace this symbols with space"""
    restricted_symbols = [
        "(", 
        ")", 
        "+", 
        "-", 
        "[", 
        "]", 
        "/", 
        ".", 
        ",", 
        "@", 
        "{", 
        "}", 
        "«", 
        "»", 
        "<", 
        ">", 
        ":", 
        ";", 
        "'",
        "*",
        "?",
        "!",
        "-",
        "_",
    ]
    return "".join(c for c in col_name if not c.isalpha() and not c.isdigit() and not c.isspace() and c not in restricted_symbols)

non_alpha = df_cleansed["column_name"].progress_apply(lambda x: remove_alphabetic_symbols(str(x)))
non_alpha.unique()

100%|████████████████████████████| 4801281/4801281 [00:04<00:00, 1004772.29it/s]


array(['', '№', '%', '#', '±', '−', '=', '—', '∆%', '–', '€', '\u200e',
       '\\', '""', '$', '–%', '⋅', '\xad', '\\№', '…', '‰', '%~', '—…',
       '°', '~%', '·', '&', '|', '’', '¬', '%%%%%%%%', '••', '||', '₽',
       '⊙', '’’', '£', '———', '≡', '—%', '≈', '¥', '~', '’—', '∞',
       '\x7f"``"\x7f', '#️⃣', '▼', '%%%%%%', '⠀', '$$', '‘’', '±°', '′′',
       '（）', '↕', '→↓', '⊕', '\xad\xad', '\xad\xad\xad\xad\xad',
       '\xad\xad\xad\xad', '‡', '•', '——', '•&', '·−', '=""', '×', '°·',
       '=−', '∑', '†', '₣', '№———————№№———№—№', '▼§', '↓→', '↓', '˦', '˩',
       '˧˩', '˥˧', '˥', '%%', '\xad\xad\xad', '́', '↓·→', '’%', '№№',
       '%–', '№\\', '"', '××', '−−', '∙−', '∙∙−', '№#', '×”', '→',
       '́́́́́́́́́́́́́́́', '—#&~', '№—', 'ಂ', 'ಃ', '——————', '：', '̃', '←',
       '↔', '№№——№—№——№——№№—№—№№—', '‑', '̈̈', '°—', '%—', '±%', '&&',
       '्', '̄̄', '·°', '№&', '•—', '•••', '₨', '½', '¼', '———————', '−%',
       '#&', 'ꤦ', 'ꤧ', 'ꤨ', 'ꤩ', 'ꤪ', '∨', '́́́́́́́́́́', '÷%', '≤°', '≥°

In [45]:
# # - тоже номер или не всегда
# Много знаков равенства, скорее всего в столбце число лежит, можно тоже указать
rules = {
    "№": " номер ",
    "%": " процент ",
    "∆": " число ",
    "∑": " сумма ",
    "Σ": " сумма ",
    "$": " деньги ",
    "₽": " деньги ",
    "€": " деньги ",
    "¥": " деньги ",
    "₩": " деньги ",
    "£": " деньги ",
}

for rule, word in rules.items():
    df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).replace(rule, word))

df_cleansed

100%|████████████████████████████| 4801281/4801281 [00:01<00:00, 2699082.06it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,Конструктор
1,dataset/Dataset/data/3693708/table_0,1,Двигатель
2,dataset/Dataset/data/3693708/table_0,2,Сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,Лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,Очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,Дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,Сезон
415129,dataset/Dataset/data/6206410/table_0,1,Короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3.1. To lower

In [46]:
df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).lower())
df_cleansed

100%|████████████████████████████| 4801281/4801281 [00:02<00:00, 2281691.60it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,конструктор
1,dataset/Dataset/data/3693708/table_0,1,двигатель
2,dataset/Dataset/data/3693708/table_0,2,сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,сезон
415129,dataset/Dataset/data/6206410/table_0,1,короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,произвольная программа


## 3.2 замена ё на е

In [47]:
df_cleansed["column_name"] = df_cleansed["column_name"].str.replace("ё", "е")

## 4. Удаление символов не из русского алфавита

In [48]:
# for i in df_cleansed.sort_values(by=["column_name"], ascending=False)["column_name"]:
#     print(i)

In [49]:
def remove_non_alphabetic_symbols(col_name: str) -> str:
    # return "".join(c for c in col_name if c.isalpha() or c == " ")
    # return "".join(c for c in col_name if "А" <= c <= "я" or c == " ")
    for c in col_name:
        if not ("а" <= c <= "я" or c == " "):
            col_name = col_name.replace(c, " ")
    return col_name
    # return "".join(c for c in col_name if "а" <= c <= "я" or c == " ")

df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: remove_non_alphabetic_symbols(str(x)))
df_cleansed

100%|████████████████████████████| 4801281/4801281 [00:03<00:00, 1258292.46it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,конструктор
1,dataset/Dataset/data/3693708/table_0,1,двигатель
2,dataset/Dataset/data/3693708/table_0,2,сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,сезон
415129,dataset/Dataset/data/6206410/table_0,1,короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [50]:
len_le_2 = df_cleansed["column_name"].str.len() == 0
df_cleansed[len_le_2]

,table_id,column_id,column_name
79795,dataset/Dataset/data/8289176/table_1,7,
379229,dataset/Dataset/data/8113812/table_1,7,


## 5. Удаление пробелов с начала и конца заголовка столбца

In [51]:
df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).strip())
df_cleansed = df_cleansed[df_cleansed["column_name"] != ""]

100%|████████████████████████████| 4801281/4801281 [00:01<00:00, 2761701.92it/s]


## 6. Удаление заголовков столбцов, используемых для группировки значений (по месяцу, году и т.п.)

In [52]:
group_by_columns = [
    "январь",
    "февраль",
    "март",
    "апрель",
    "май",
    "июнь",
    "июль",
    "август",
    "сентябрь",
    "октябрь",
    "ноябрь",
    "декабрь",
    "янв",
    "февр",
    "фев",
    "мар",
    "апр",
    "июн",
    "июл",
    "авг",
    "сент",
    "сен",
    "окт",
    "ноя",
    "нояб",
    "дек",
]

In [53]:
s = 0
for col in group_by_columns:
    s += df_cleansed[df_cleansed["column_name"] == col].shape[0]
s

137389

In [54]:
for col in group_by_columns:
    group_by_mask = df_cleansed["column_name"] == col
    df_cleansed = df_cleansed[~group_by_mask]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,конструктор
1,dataset/Dataset/data/3693708/table_0,1,двигатель
2,dataset/Dataset/data/3693708/table_0,2,сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,сезон
415129,dataset/Dataset/data/6206410/table_0,1,короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [55]:
(df_cleansed["column_name"] == "фев").sum()

0

## 7. Удаление заголовков длиной <= 2

TODO: split count max len of word; if <= 2 -> drop

In [56]:

len_le_2 = df_cleansed["column_name"].str.len() <= 2
df_cleansed[len_le_2]

,table_id,column_id,column_name
83,dataset/Dataset/data/7901984/table_1,2,и
84,dataset/Dataset/data/7901984/table_1,3,в
85,dataset/Dataset/data/7901984/table_1,4,вп
86,dataset/Dataset/data/7901984/table_1,5,пп
87,dataset/Dataset/data/7901984/table_1,6,п
...,...,...,...
415114,dataset/Dataset/data/6418067/table_2,3,в
415115,dataset/Dataset/data/6418067/table_2,4,н
415116,dataset/Dataset/data/6418067/table_2,5,п
415117,dataset/Dataset/data/6418067/table_2,6,гз


In [57]:
df_cleansed = df_cleansed[~len_le_2]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,конструктор
1,dataset/Dataset/data/3693708/table_0,1,двигатель
2,dataset/Dataset/data/3693708/table_0,2,сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,очки
...,...,...,...
415127,dataset/Dataset/data/35185/table_10,6,дата выхода из союза
415128,dataset/Dataset/data/6206410/table_0,0,сезон
415129,dataset/Dataset/data/6206410/table_0,1,короткая программа
415130,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [58]:
df_cleansed = df_cleansed.reset_index().drop(["index"], axis=1)
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/3693708/table_0,0,конструктор
1,dataset/Dataset/data/3693708/table_0,1,двигатель
2,dataset/Dataset/data/3693708/table_0,2,сезоны участия
3,dataset/Dataset/data/3693708/table_0,3,лучшийрезультат
4,dataset/Dataset/data/3693708/table_0,4,очки
...,...,...,...
3381729,dataset/Dataset/data/35185/table_10,6,дата выхода из союза
3381730,dataset/Dataset/data/6206410/table_0,0,сезон
3381731,dataset/Dataset/data/6206410/table_0,1,короткая программа
3381732,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [117]:
from random import randint

rnd = randint(0, df_cleansed.shape[0])
df_cleansed.loc[rnd:rnd+10]

,table_id,column_id,column_name
129321,dataset/Dataset/data/979686/table_0,3,лауреаты
129322,dataset/Dataset/data/979686/table_0,4,результаты
129323,dataset/Dataset/data/1825597/table_0,0,место
129324,dataset/Dataset/data/1825597/table_0,1,спортсмен
129325,dataset/Dataset/data/1825597/table_0,2,страна
129326,dataset/Dataset/data/1825597/table_0,3,время
129327,dataset/Dataset/data/8169719/table_0,0,место
129328,dataset/Dataset/data/8169719/table_0,1,компания
129329,dataset/Dataset/data/8169719/table_0,2,доход за г млн долл
129330,dataset/Dataset/data/4780282/table_0,0,год


# Сохранение очищенного датафрейма

In [60]:
import numpy as np

number_of_chunks = 10
for id, chunk in enumerate(np.array_split(df_cleansed, number_of_chunks)):
    chunk.to_csv(f"./cleansed/df_cleansed_{id}.csv", sep=";", index=False)

/home/kirilltobola/Dev/github.com/isp/cta/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [61]:
# df_cleansed.to_csv("df_cleansed.csv", sep=";", index=False)